In [3]:
from mainnet_launch.pages.risk_metrics.exit_liqudity_quotes import (
    _pick_a_quote_batch,
    _load_full_quote_batch_df,
    _augment_swap_quotes_df,
)

import plotly.express as px
import streamlit as st
import plotly.io as pio
import pandas as pd

pio.templates.default = "plotly_white"

from mainnet_launch.constants import *

from mainnet_launch.database.schema.full import Destinations, AutopoolDestinations, Tokens, SwapQuote
from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_df,
    get_highest_value_in_field_where,
)

from mainnet_launch.pages.risk_metrics.drop_down import render_pick_chain_and_base_asset_dropdown


swap_quotes_df = get_full_table_as_df(
    SwapQuote,
)

# _augment_swap_quotes_df(swap_quotes_df)
swap_quotes_df

,chain_id,base_asset,api_name,sell_token_address,buy_token_address,scaled_amount_in,scaled_amount_out,pools_blacklist,percent_exclude_threshold,aggregator_name,datetime_received,quote_batch,id
0,8453,0x4200000000000000000000000000000000000006,odos,0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22,0x4200000000000000000000000000000000000006,375.0,4.134614e+02,"['0xb7b8b3afc010169779c5c2385ec0eb0477fe3347',...",10.0,Odos,2025-08-12 19:37:18.059929+00:00,0,2757a036-6a69-4894-9a75-09d3330716d9
1,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x15700B564Ca08D9439C58cA5053166E8317aa138,0x865377367054516e17014CcdED1e7d814EDC9ce4,400000.0,4.006290e+05,"['0x6691dbb44154a9f23f8357c56fc9ff5548a8bdc4',...",10.0,Odos,2025-08-12 19:35:29.938495+00:00,0,1ff8557b-c5c4-4be8-ac2f-20ff5d810729
2,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,275.0,4.797961e+01,"['0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5',...",10.0,Odos,2025-08-12 19:33:45.758044+00:00,0,8dacd74d-7361-4bd7-8e7a-6ffe18733429
3,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,75.0,7.904743e+01,None,10.0,Odos,2025-08-12 19:34:24.562109+00:00,0,ba159138-9908-47af-9c71-686afb83ca75
4,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0x57aB1E0003F623289CD798B1824Be09a793e4Bec,0x865377367054516e17014CcdED1e7d814EDC9ce4,800000.0,7.926348e+05,None,10.0,Odos,2025-08-12 19:36:26.593524+00:00,0,74eac004-5334-4d70-8263-264586e0b0d8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70225,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0x865377367054516e17014CcdED1e7d814EDC9ce4,900000.0,1.070681e+06,None,99.0,Odos,2025-08-15 14:31:26.751341+00:00,47,de3b59aa-8ce7-484d-8572-b8c261e428c6
70226,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0x865377367054516e17014CcdED1e7d814EDC9ce4,200000.0,2.379412e+05,None,99.0,Odos,2025-08-15 14:32:37.972229+00:00,47,5737b458-3abc-454d-919d-389f0c4a8ce9
70227,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0x865377367054516e17014CcdED1e7d814EDC9ce4,700000.0,8.351777e+05,None,99.0,4626,2025-08-15 14:32:23.970367+00:00,47,a57ac628-5246-4e33-a392-26ba0f224d3a
70228,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,75.0,9.063000e+01,None,99.0,Odos,2025-08-15 14:40:27.493884+00:00,47,17a96f82-b7d1-4ea1-bf7f-f295241358ae


In [8]:
swap_quotes_df["day"] = swap_quotes_df["datetime_received"].dt.date

In [9]:
swap_quotes_df["day"].value_counts()

day
2025-08-15    23147
2025-08-14    21463
2025-08-13    21110
2025-08-12     4510
Name: count, dtype: int64

In [ ]:
for api_name in ["odos", "tokemak"]:
    local_df = swap_quotes_df[swap_quotes_df["api_name"] == api_name].copy()
    local_df = local_df.sort_values(["label", "scaled_amount_in"])

    fig = px.line(
        local_df,
        x="scaled_amount_in",
        y="slippage_bps",
        color="label",
        markers=True,  # adds markers
    )

    fig.update_traces(
        mode="lines+markers",  # show both
        line=dict(dash="dash"),  # dashed lines
        marker_symbol="x",  # 'x marks the spot'
        marker_size=8,
    )

    fig.update_layout(
        title=f"{api_name}: Slippage vs Amount Sold",
        xaxis_title="scaled_amount_in",
        yaxis_title="slippage_bps",
    )
    fig.show()

In [40]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

# Sorted symbols
symbol_options = sorted(swap_quotes_df["sell_token_symbol"].dropna().unique().tolist())
factors = ["absolute", "portion"]
factor_symbol = {"absolute": "circle", "portion": "x"}

# Stable colors per label across all charts
all_labels = sorted(swap_quotes_df["label"].dropna().unique().tolist())
palette = px.colors.qualitative.Plotly
label_color = {lbl: palette[i % len(palette)] for i, lbl in enumerate(all_labels)}

for selected_symbol in symbol_options:
    fig = go.Figure()

    for size_factor in factors:
        df_factor = (
            swap_quotes_df[
                (swap_quotes_df["sell_token_symbol"] == selected_symbol)
                & (swap_quotes_df["size_factor"] == size_factor)
            ]
            .groupby(["label", "scaled_amount_in"], as_index=False)["effective_price"]
            .median()
        )
        if df_factor.empty:
            continue

        # One trace per (label, factor) to overlay both factors
        for label, df_lab in df_factor.groupby("label", sort=True):
            fig.add_trace(
                go.Scatter(
                    x=df_lab["scaled_amount_in"],
                    y=df_lab["effective_price"],
                    mode="markers",
                    name=f"{label} ({size_factor})",
                    legendgroup=label,  # group legend entries by label
                    marker=dict(
                        symbol=factor_symbol.get(size_factor, "circle"),
                        size=7,
                        color=label_color.get(label),
                        line=dict(width=0.5),
                    ),
                    hovertemplate=(
                        "label=%{text}<br>"
                        "factor=" + size_factor + "<br>"
                        "scaled_amount_in=%{x}<br>"
                        "effective_price=%{y}<extra></extra>"
                    ),
                    text=[label] * len(df_lab),
                )
            )

    fig.update_xaxes(title_text="scaled_amount_in")
    fig.update_yaxes(title_text="effective_price")
    fig.update_layout(
        title_text=f"Slippage vs Amount — {selected_symbol} (absolute & portion overlaid)",
        margin=dict(t=60, r=20, b=20, l=50),
        legend_title_text="label (factor)",
    )

    # st.plotly_chart(fig, use_container_width=True)
    fig.show()